# Cross-linked Polymer Networks

This tutorial shows how to create cross-linked networks using LAMMPS reactive MD.

## Overview

We'll:
1. Create EO2 (linear) and EO3 (branched) monomers
2. Generate reaction templates for all combinations
3. Pack monomers into initial configuration
4. Set up LAMMPS reactive MD with fix bond/react

## Chemistry

**Reaction**: Dehydration (ether formation)
- R-OH + HO-R' → R-O-R' + H₂O

**Monomers**:
- EO2: Linear, 2 reactive sites
- EO3: Branched, 3 reactive sites (enables network)

## Why Cross-linking?

Creates thermoset materials with:
- Enhanced strength
- Chemical resistance
- Dimensional stability

# Case 3: Cross-linked System using TemplateReacter

This notebook implements a cross-linked polymer system using:
- EO2 (ethylene oxide dimer) and EO3 (ethylene oxide trimer) monomers
- TemplateReacter for cross-linking reactions
- LAMMPS fix bond/react templates

**Key features:**
- Builds EO2 and EO3 monomers
- Generates reaction templates for all monomer combinations (EO2+EO2, EO2+EO3, EO3+EO3)
- Creates initial configuration using packmol (mixed EO2/EO3 system)
- Generates LAMMPS input script for reactive MD

## Step 1: Import Required Libraries

Import all necessary modules from MolPy for building cross-linked polymer systems, handling reactions, and exporting to LAMMPS format.

## Reaction Templates

Generate LAMMPS `fix bond/react` templates for all monomer pairs:
- **EO2-EO2**: Linear chain extension
- **EO2-EO3**: Branch point formation
- **EO3-EO3**: Dense crosslinking

Each template specifies:
- Reactant structures (before)
- Product structure (after)
- Atom mapping
- Leaving groups (H₂O)

LAMMPS will use these to detect and execute reactions during MD.

## Initial Configuration

Pack monomers into simulation box:
- **Random positions**: Avoid overlaps
- **Target density**: Realistic polymer melt
- **Box size**: Calculated from mass and density

This creates the starting configuration for reactive MD.

In [ ]:
import numpy as np
import molpy as mp
from molpy.core.atomistic import Atom, Atomistic, Bond
from molpy.core.entity import Entity
from molpy.core.frame import Frame
from molpy.core.box import Box
from molpy.core.forcefield import ForceField
from molpy.external import RDKitAdapter, Generate3D
from molpy.parser.smiles import parse_bigsmiles, bigsmilesir_to_monomer
from molpy.reacter import (
    select_identity,
    select_c_neighbor,
    select_hydroxyl_group,
    select_hydroxyl_h_only,
    form_single_bond,
    find_port_atom,
)
from molpy.reacter.template import TemplateReacter, TemplateResult, write_template_files
from molpy.typifier.atomistic import OplsAtomisticTypifier
from molpy.io.data.lammps import LammpsDataWriter
from molpy.io.forcefield.lammps import LAMMPSForceFieldWriter
from molpy.pack import InsideBoxConstraint, Molpack
from pathlib import Path
from typing import List, Dict, Set

## Step 2: Load Force Field

Load the OPLS-AA force field and create a typifier.

In [ ]:
# Load force field
forcefield_path = "oplsaa.xml"
ff = mp.io.read_xml_forcefield(forcefield_path)
typifier = OplsAtomisticTypifier(ff, strict_typing=False)

In [ ]:
print("✅ Force field loaded successfully")

## Step 3: Define Helper Functions

Define helper functions for:
- Monomer builders: Build EO2 monomer
- Reaction runners: Execute dehydration reactions using TemplateReacter

In [ ]:
def build_monomer(bigsmiles: str, typifier: OplsAtomisticTypifier) -> Atomistic:
    """Build a monomer from BigSMILES string with 3D coordinates and types."""
    ir = parse_bigsmiles(bigsmiles)
    monomer = bigsmilesir_to_monomer(ir)
    
    adapter = RDKitAdapter(internal=monomer)
    generate_3d = Generate3D(add_hydrogens=True, embed=True, optimize=True, update_internal=True)
    adapter = generate_3d(adapter)
    monomer = adapter.get_internal()
    
    monomer.get_topo(gen_angle=True, gen_dihe=True)
    
    for idx, atom in enumerate(monomer.atoms):
        atom["id"] = idx + 1
    
    typifier.typify(monomer)
    
    return monomer

### Generating Reaction Templates

Creates LAMMPS fix bond/react templates for all monomer pair combinations.

In [ ]:
def run_reaction_with_template(
    left: Atomistic,
    right: Atomistic,
    port_L: str,
    port_R: str,
    reaction_name: str = "crosslinking",
    radius: int = 6,
) -> TemplateResult:
    """Run dehydration reaction and generate template using TemplateReacter.
    
    This function:
    1. Creates a TemplateReacter with the dehydration reaction settings
    2. Runs the reaction and generates template
    3. Returns the template result
    
    Note: If both left and right have the same port name (e.g., both have '$'),
    this function will automatically select different ports:
    - port_L will use the rightmost port (higher atom id)
    - port_R will use the leftmost port (lower atom id)
    """
    # Create TemplateReacter directly with Reacter constructor parameters
    template_reacter = TemplateReacter(
        name=reaction_name,
        anchor_selector_left=select_c_neighbor,
        anchor_selector_right=select_identity,
        leaving_selector_left=select_hydroxyl_group,
        leaving_selector_right=select_hydroxyl_h_only,
        bond_former=form_single_bond,
        radius=radius,
    )
    
    # Find port atoms
    # Handle case where both monomers have the same port name
    if port_L == port_R and port_L in [a.get("port") for a in left.atoms] and port_L in [a.get("port") for a in right.atoms]:
        # Find all atoms with this port name
        left_ports = [a for a in left.atoms if a.get("port") == port_L]
        right_ports = [a for a in right.atoms if a.get("port") == port_R]
        
        if len(left_ports) >= 2 and len(right_ports) >= 1:
            # For left: use the rightmost port (higher atom id) for port_L
            # For right: use the leftmost port (lower atom id) for port_R
            port_atom_L = max(left_ports, key=lambda a: a.get("id", 0))
            port_atom_R = min(right_ports, key=lambda a: a.get("id", 0))
        elif len(left_ports) >= 1 and len(right_ports) >= 2:
            # For left: use the leftmost port (lower atom id) for port_L
            # For right: use the rightmost port (higher atom id) for port_R
            port_atom_L = min(left_ports, key=lambda a: a.get("id", 0))
            port_atom_R = max(right_ports, key=lambda a: a.get("id", 0))
        else:
            # Fallback to standard behavior
            port_atom_L = find_port_atom(left, port_L)
            port_atom_R = find_port_atom(right, port_R)
    else:
        # Standard case: different port names or only one port per monomer
        port_atom_L = find_port_atom(left, port_L)
        port_atom_R = find_port_atom(right, port_R)
    
    # Run reaction with template generation
    result, template = template_reacter.run_with_template(
        left=left,
        right=right,
        port_atom_L=port_atom_L,
        port_atom_R=port_atom_R,
        compute_topology=True,
        record_intermediates=False,
    )
    
    return template

In [ ]:
print("✅ Helper functions defined")

## Step 4: Build EO2 and EO3 Monomers

Build the EO2 (ethylene oxide dimer) and EO3 (ethylene oxide trimer) monomers from BigSMILES notation.

In [ ]:
def build_eo2_monomer(typifier: OplsAtomisticTypifier) -> Atomistic:
    """Build EO2 monomer: HO-CH2CH2-O-CH2CH2-OH"""
    bigsmiles = '{[][$]OCCOCCOCCO[$][]}'
    return build_monomer(bigsmiles, typifier)

In [ ]:
def build_eo3_monomer(typifier: OplsAtomisticTypifier) -> Atomistic:
    """Build EO3 monomer: 3-arm branched structure C(COCCO[$])(COCCO[$])COCCO[$]"""
    # 3-arm structure: center C with three COCCO branches, each with a [$] port
    # Note: BigSMILES requires [<] at the start, so we add it to the center C
    bigsmiles = '{[]C(COCCO[$])(COCCO[$])COCCO[$][]}'
    return build_monomer(bigsmiles, typifier)

In [ ]:
# Build EO2 and EO3 monomers
eo2_monomer = build_eo2_monomer(typifier)
eo3_monomer = build_eo3_monomer(typifier)

In [ ]:
eo2_ports = [atom.get("port") for atom in eo2_monomer.atoms if atom.get("port") is not None]
eo3_ports = [atom.get("port") for atom in eo3_monomer.atoms if atom.get("port") is not None]

### Output Information

Displaying system statistics and verification.

In [ ]:
print(f"✅ EO2 monomer built:")
print(f"   Atoms: {len(eo2_monomer.atoms)}")
print(f"   Ports: {eo2_ports}")

### Output Information

Displaying system statistics and verification.

In [ ]:
print(f"✅ EO3 monomer built:")
print(f"   Atoms: {len(eo3_monomer.atoms)}")
print(f"   Ports: {eo3_ports}")

## Step 5: Generate Reaction Templates

Generate LAMMPS fix bond/react templates for EO2 + EO2, EO2 + EO3, and EO3 + EO3 reactions.

In [ ]:
# Create output directory
output_dir = Path("case3_output")
output_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
# Generate reaction templates
templates = []

### Output Information

Displaying system statistics and verification.

In [ ]:
# Template 1: EO2 + EO2
print("\nGenerating reaction template: EO2 + EO2...")
template1 = run_reaction_with_template(
    left=eo2_monomer.copy(),
    right=eo2_monomer.copy(),
    port_L="$",
    port_R="$",
    reaction_name="eo2_eo2",
    radius=4,  # Topological distance from anchor atoms to edge
)
write_template_files(
    base_path=output_dir / "rxn1",
    template=template1,
    typifier=typifier,
)
templates.append(("rxn1", template1))
print(f"✅ Reaction template rxn1 generated")

In [ ]:
# Template 2: EO2 + EO3
print("\nGenerating reaction template: EO2 + EO3...")
template2 = run_reaction_with_template(
    left=eo2_monomer.copy(),
    right=eo3_monomer.copy(),
    port_L="$",  # EO2的右侧port
    port_R="$",  # EO3的[>] port（三个分支O中的一个）
    reaction_name="eo2_eo3",
    radius=4,
)
write_template_files(
    base_path=output_dir / "rxn2",
    template=template2,
    typifier=typifier,
)
templates.append(("rxn2", template2))
print(f"✅ Reaction template rxn2 generated")

In [ ]:
# Template 3: EO3 + EO2 (reverse of EO2 + EO3)
print("\nGenerating reaction template: EO3 + EO2...")
template3 = run_reaction_with_template(
    left=eo3_monomer.copy(),
    right=eo2_monomer.copy(),
    port_L="$",  # EO3的[>] port（三个分支O中的一个）
    port_R="$",  # EO2的左侧port
    reaction_name="eo3_eo2",
    radius=4,
)
write_template_files(
    base_path=output_dir / "rxn3",
    template=template3,
    typifier=typifier,
)
templates.append(("rxn3", template3))
print(f"✅ Reaction template rxn3 generated")

In [ ]:
# Template 4: EO3 + EO3
print("\nGenerating reaction template: EO3 + EO3...")
template4 = run_reaction_with_template(
    left=eo3_monomer.copy(),
    right=eo3_monomer.copy(),
    port_L="$",  # EO3的[>] port（三个分支O中的一个）
    port_R="$",  # EO3的[>] port（三个分支O中的一个）
    reaction_name="eo3_eo3",
    radius=4,
)
write_template_files(
    base_path=output_dir / "rxn4",
    template=template4,
    typifier=typifier,
)
templates.append(("rxn4", template4))
print(f"✅ Reaction template rxn4 generated")

### Processing Loop

Iterating through configurations to set up the system.

In [ ]:
# Print summary for all templates
print(f"\n✅ All reaction templates generated:")
for rxn_name, template in templates:
    print(f"   - {rxn_name}_pre.mol, {rxn_name}_post.mol, {rxn_name}.map")
    print(f"     Pre: {len(list(template.pre.atoms))} atoms, Post: {len(list(template.post.atoms))} atoms")

### Processing Loop

Iterating through configurations to set up the system.

In [ ]:
# Load template frames for type collection
template_frames = []
for rxn_name, template in templates:
    pre_frame = template.pre.to_frame()
    post_frame = template.post.to_frame()
    template_frames.extend([pre_frame, post_frame])

## Step 6: Create Initial Configuration

Create initial LAMMPS configuration by packing EO2 monomers into a simulation box using packmol. Calculate box size from target density.

In [ ]:
def calculate_molecular_weight(monomer: Atomistic) -> float:
    """Calculate molecular weight of a monomer."""
    from molpy.core.element import Element
    total_mw = 0.0
    for atom in monomer.atoms:
        symbol = atom.get("symbol", "C")
        symbol_upper = symbol.upper() if symbol else "C"
        element = Element(symbol_upper)
        total_mw += element.mass
    return total_mw

### Setting System Density

Target density determines box size. Typical polymer melts: 0.8-1.2 g/cm³.

In [ ]:
def calculate_box_size_from_density(
    n_monomers: int,
    monomer: Atomistic,
    target_density: float = 1.0,
) -> float:
    """Calculate box size from target density."""
    mw = calculate_molecular_weight(monomer)
    total_mw = n_monomers * mw
    
    NA = 6.022e23
    total_mass_g = total_mw / NA
    volume_cm3 = total_mass_g / target_density
    volume_angstrom3 = volume_cm3 * 1e24
    box_length_angstrom = volume_angstrom3 ** (1.0 / 3.0)
    
    return box_length_angstrom

### Processing Loop

Iterating through configurations to set up the system.

In [ ]:
def collect_types_from_frames(*frames: Frame) -> Dict[str, Set[str]]:
    """Collect all type names from multiple frames."""
    atom_types: Set[str] = set()
    bond_types: Set[str] = set()
    angle_types: Set[str] = set()
    dihedral_types: Set[str] = set()
    
    for frame in frames:
        if "atoms" in frame and "type" in frame["atoms"]:
            for atom_type in frame["atoms"]["type"]:
                if atom_type:
                    type_str = str(atom_type)
                    if not type_str.isdigit():
                        atom_types.add(type_str)
        
        if "bonds" in frame and "type" in frame["bonds"]:
            for bond_type in frame["bonds"]["type"]:
                if bond_type:
                    type_str = str(bond_type)
                    if not type_str.isdigit():
                        bond_types.add(type_str)
        
        if "angles" in frame and "type" in frame["angles"]:
            for angle_type in frame["angles"]["type"]:
                if angle_type:
                    type_str = str(angle_type)
                    if not type_str.isdigit():
                        angle_types.add(type_str)
        
        if "dihedrals" in frame and "type" in frame["dihedrals"]:
            for dihedral_type in frame["dihedrals"]["type"]:
                if dihedral_type:
                    type_str = str(dihedral_type)
                    if not type_str.isdigit():
                        dihedral_types.add(type_str)
    
    return {
        "atom_types": atom_types,
        "bond_types": bond_types,
        "angle_types": angle_types,
        "dihedral_types": dihedral_types,
    }

### Setting System Density

Target density determines box size. Typical polymer melts: 0.8-1.2 g/cm³.

In [ ]:
# Pack monomers (mix of EO2 and EO3)
n_eo2 = 27
n_eo3 = 9
n_monomers = n_eo2 + n_eo3
target_density = 1.2

In [ ]:
# Calculate average molecular weight for box size calculation
mw_eo2 = calculate_molecular_weight(eo2_monomer)
mw_eo3 = calculate_molecular_weight(eo3_monomer)
avg_mw = (n_eo2 * mw_eo2 + n_eo3 * mw_eo3) / n_monomers
total_mw = n_eo2 * mw_eo2 + n_eo3 * mw_eo3

### Setting System Density

Target density determines box size. Typical polymer melts: 0.8-1.2 g/cm³.

In [ ]:
NA = 6.022e23
total_mass_g = total_mw / NA
volume_cm3 = total_mass_g / target_density
volume_angstrom3 = volume_cm3 * 1e24
box_length = volume_angstrom3 ** (1.0 / 3.0)

In [ ]:
print(f"\n✅ Calculated box size: {box_length:.2f} Å")
print(f"   Mix: {n_eo2} EO2 + {n_eo3} EO3 = {n_monomers} total monomers")

### Processing Loop

Iterating through configurations to set up the system.

In [ ]:
# Helper function to clean port metadata before packing
def clean_port_metadata(monomer: Atomistic) -> Atomistic:
    """Remove port metadata fields to avoid packing inconsistencies."""
    cleaned = monomer.copy()
    port_fields = ['port', 'port_role', 'port_bond_kind', 'port_compat', 'port_priority', 'ports']
    for atom in cleaned.atoms:
        for field in port_fields:
            if field in atom.data:
                del atom.data[field]
    return cleaned

In [ ]:
# Prepare frames for packing (clean port metadata first)
eo2_clean = clean_port_metadata(eo2_monomer)
eo3_clean = clean_port_metadata(eo3_monomer)
eo2_frame = eo2_clean.to_frame()
eo3_frame = eo3_clean.to_frame()

In [ ]:
if "charge" in eo2_frame["atoms"]:
    eo2_frame["atoms"]["q"] = eo2_frame["atoms"]["charge"]
if "charge" in eo3_frame["atoms"]:
    eo3_frame["atoms"]["q"] = eo3_frame["atoms"]["charge"]

## Initial Configuration

Pack monomers into simulation box:
- **Random positions**: Avoid overlaps
- **Target density**: Realistic polymer melt
- **Box size**: Calculated from mass and density

This creates the starting configuration for reactive MD.

In [ ]:
# Use packmol for packing
pack_workdir = output_dir / "packmol_work"
packer = Molpack(workdir=pack_workdir)

In [ ]:
origin = np.array([0.0, 0.0, 0.0])
length = np.array([box_length, box_length, box_length])
box_constraint = InsideBoxConstraint(length=length, origin=origin)

In [ ]:
packer.add_target(eo2_frame, number=n_eo2, constraint=box_constraint)
packer.add_target(eo3_frame, number=n_eo3, constraint=box_constraint)

In [ ]:
print(f"   Packing {n_eo2} EO2 + {n_eo3} EO3 monomers...")
packed_frame = packer.optimize(max_steps=10000, seed=42)

### Output Information

Displaying system statistics and verification.

In [ ]:
print(f"✅ Packed system: {packed_frame['atoms'].nrows} atoms")

In [ ]:
# Ensure required fields
if "charge" in packed_frame["atoms"] and "q" not in packed_frame["atoms"]:
    packed_frame["atoms"]["q"] = packed_frame["atoms"]["charge"]
elif "q" in packed_frame["atoms"] and "charge" not in packed_frame["atoms"]:
    packed_frame["atoms"]["charge"] = packed_frame["atoms"]["q"]

In [ ]:
# Create box object
box = Box.cubic(length=box_length)
packed_frame.metadata["box"] = box

In [ ]:
# Collect types from config and templates
all_frames = [packed_frame] + template_frames
types_from_config = collect_types_from_frames(*all_frames)

In [ ]:
# Add type labels to frame metadata
packed_frame.metadata["type_labels"] = {
    "atom_types": sorted(list(types_from_config["atom_types"])),
    "bond_types": sorted(list(types_from_config["bond_types"])),
    "angle_types": sorted(list(types_from_config["angle_types"])),
    "dihedral_types": sorted(list(types_from_config["dihedral_types"])),
}

In [ ]:
# Write force field file
ff_path = output_dir / "case3_crosslinking.ff"
ff_writer = LAMMPSForceFieldWriter(ff_path)
ff_writer.write(
    ff,
    atom_types=types_from_config["atom_types"],
    bond_types=types_from_config["bond_types"],
    angle_types=types_from_config["angle_types"],
    dihedral_types=types_from_config["dihedral_types"],
)

In [ ]:
# Write data file
data_path = output_dir / "case3_crosslinking.data"
data_writer = LammpsDataWriter(data_path, atom_style="full")
data_writer.write(packed_frame)

In [ ]:
print(f"✅ Written: {ff_path.name}, {data_path.name}")

## Step 7: Generate LAMMPS Input Script

Generate LAMMPS input script for reactive MD simulation using fix bond/react.

In [ ]:
# Build molecule commands
molecule_commands = []
react_commands = []

### Processing Loop

Iterating through configurations to set up the system.

In [ ]:
for rxn_name, _ in templates:
    molecule_commands.append(f"molecule   {rxn_name}_pre {rxn_name}_pre.mol")
    molecule_commands.append(f"molecule   {rxn_name}_post {rxn_name}_post.mol")
    react_commands.append(f"  react {rxn_name} all 1 0.0 5 {rxn_name}_pre {rxn_name}_post {rxn_name}.map prob 0.2 5123 rescale_charges yes &")

### Setting System Density

Target density determines box size. Typical polymer melts: 0.8-1.2 g/cm³.

In [ ]:
script_content = f"""# LAMMPS input script for Case 3: Cross-linked EO2/EO3 System
# Force field: OPLS-AA
# Reaction: Dehydration to form ether bonds
# Monomers: {n_eo2} EO2 + {n_eo3} EO3
#
# ============================================================================
# Initialization
# ============================================================================
#
units           real
atom_style      full
boundary        p p p
dimension       3

# ============================================================================
# Read data and force field
# ============================================================================
#
read_data       case3_crosslinking.data &
    extra/bond/per/atom 25 &
    extra/angle/per/atom 25 &
    extra/dihedral/per/atom 25 &
    extra/improper/per/atom 25 &
    extra/special/per/atom 25
include         case3_crosslinking.ff
kspace_style    pppm 1.0e-5
special_bonds   lj/coul 0.0 0.0 0.5

# ============================================================================
# Neighbor settings
# ============================================================================
#
neighbor        2.0 bin
neigh_modify    delay 0 every 1 check yes

# ============================================================================
# Energy minimization
# ============================================================================
#
print "=========================================="
print "Step 1: Energy Minimization"
print "=========================================="

minimize        1.0e-4 1.0e-4 1000 10000

# ============================================================================
# Equilibration: NPT
# ============================================================================
#
print "=========================================="
print "Step 2: NPT Equilibration"
print "=========================================="

velocity all create 300.0 1234

timestep        0.1

fix             shake all shake 0.0001 20 0 t opls_140 opls_155 opls_185

fix             npt all npt temp 300.0 300.0 100.0 iso 1.5 1.5 1000.0

dump            2 all custom 500 equil.dump id type x y z

run             1000
unfix           npt
undump          2

# ============================================================================
# Reactive MD: fix bond/react
# ============================================================================
#
print "=========================================="
print "Step 3: Reactive MD with fix bond/react"
print "=========================================="

# Read reaction templates
{chr(10).join(molecule_commands)}

fix rxns all bond/react stabilization yes npt_grp .03 &
{chr(10).join(react_commands)}
    # End of reactions

unfix shake

# Thermostat and barostat for reactive MD
fix             npt_grp_react all npt temp 300.0 300.0 100.0 iso 1.5 1.5 1000.0

# Output settings
thermo          100
thermo_style    custom elapsed temp pe ke etotal press vol density f_rxns[*]
thermo_modify   flush yes

# Dump trajectory
compute 1 all property/local batom1 batom2 btype
dump            3 all custom 10 react.dump id mol type xu yu zu
dump            4 all local 10 topo.dump index c_1[1] c_1[2] c_1[3]
run             5000

undump         3
undump         4
unfix          rxns
unfix          npt_grp_react

# ============================================================================
# Production MD
# ============================================================================
#
print "=========================================="
print "Step 4: Production MD"
print "=========================================="

thermo_style    custom elapsed temp pe ke etotal press vol density

# ============================================================================
# Final output
# ============================================================================
#
write_data      final_crosslinked.data
write_restart   final_crosslinked.restart

print "=========================================="
print "Simulation completed!"
print "=========================================="
"""

In [ ]:
script_path = output_dir / "run_case3_crosslinking.in"
with script_path.open("w") as f:
    f.write(script_content)

In [ ]:
print(f"✅ Generated LAMMPS input script: {script_path.name}")

## Summary

This notebook demonstrated the complete workflow for building a cross-linked polymer system:

1. ✅ Loaded OPLS-AA force field
2. ✅ Built EO2 and EO3 monomers
3. ✅ Generated reaction templates for LAMMPS fix bond/react using TemplateReacter:
   - EO2 + EO2 (rxn1)
   - EO2 + EO3 (rxn2)
   - EO3 + EO2 (rxn3)
   - EO3 + EO3 (rxn4)
4. ✅ Created initial configuration using packmol (mixed EO2/EO3 system)
5. ✅ Generated LAMMPS input script for reactive MD

The generated files can be used for:
- LAMMPS reactive MD simulations (run_case3_crosslinking.in)
- Reaction templates (rxn1-4: pre.mol, post.mol, map files)
- Initial configuration (case3_crosslinking.data, case3_crosslinking.ff)